In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image

In [2]:
# === Set paths ===
project_dir = os.path.dirname(os.getcwd())
sheet_path = os.path.join(project_dir, 'rendering', 'pre_choice.xlsx')  # Update if needed
output_folder = os.path.join(project_dir, 'image')
output_excel_path = os.path.join(output_folder, 'result_df_selected.xlsx')

In [3]:
result_df = pd.read_excel(sheet_path)

In [4]:
# === Define rendering functions ===
def save_problem_grid(image, filename):
    fig = plt.figure(figsize=(10, 10))
    for i in range(8):
        ax = plt.subplot(3, 3, i + 1)
        ax.imshow(image[i], cmap='gray')
        ax.axis('off')
        rect = Rectangle((0, 0), 1, 1, transform=ax.transAxes, color='gray', fill=False, lw=1.8)
        ax.add_patch(rect)
    ax = plt.subplot(3, 3, 9)
    ax.text(0.5, 0.5, '?', fontsize=40, ha='center', va='center')
    ax.axis('off')
    rect = Rectangle((0, 0), 1, 1, transform=ax.transAxes, color='gray', fill=False, lw=1.8)
    ax.add_patch(rect)
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(filename)
    plt.close(fig)

def save_choices_plot(image, target, filename):
    fig = plt.figure(figsize=(12, 6))
    for i in range(8, 16):
        ax = plt.subplot(2, 4, i - 7)
        ax.set_title(f"Choice {i - 7}")
        ax.imshow(image[i], cmap='gray')
        ax.axis('off')
        rect = Rectangle((0, 0), 1, 1, transform=ax.transAxes, color='gray', fill=False, lw=1.8)
        ax.add_patch(rect)
    plt.subplots_adjust(wspace=0.1, hspace=0.3)
    plt.savefig(filename)
    plt.close(fig)

In [5]:
# === Design folder mapping ===
design_paths = {
    'In_Out_Center_Single': 'output/in_center_single_out_center_single',
    'In_Out_Distribute_Four': 'output/in_distribute_four_out_center_single',
    'Left_Right_Single': 'output/left_center_single_right_center_single',
    'Up_Down_Single': 'output/up_center_single_down_center_single',
}


In [6]:
# === Prepare output tracking ===
records = []

In [7]:
# === Loop through each selected item ===
for idx, row in result_df.iterrows():
    new_item = row.get('New_Item')
    design = row['Design']
    file_name = new_item if isinstance(new_item, str) else None

    if not file_name or design not in design_paths:
        continue

    npz_path = os.path.join(project_dir, design_paths[design], file_name + '.npz')
    if not os.path.exists(npz_path):
        print(f"Missing file: {npz_path}")
        continue

    # === Load data and extract target ===
    data = np.load(npz_path)
    image = data['image']
    target = int(data['target'])
    target_plus_one = target + 1

    # === Render and combine ===
    temp_dir = os.path.join(project_dir, "temp_img")
    os.makedirs(temp_dir, exist_ok=True)
    problem_path = os.path.join(temp_dir, "problem_grid.png")
    choice_path = os.path.join(temp_dir, "choices_plot.png")

    save_problem_grid(image, problem_path)
    save_choices_plot(image, target, choice_path)

    grid_img = Image.open(problem_path)
    choice_img = Image.open(choice_path)

    w = max(grid_img.width, choice_img.width)
    h = grid_img.height + choice_img.height
    combined = Image.new("RGB", (w, h), (255, 255, 255))
    combined.paste(grid_img, ((w - grid_img.width) // 2, 0))
    combined.paste(choice_img, ((w - choice_img.width) // 2, grid_img.height))

    output_name = f"{row['File_Name']}_val.png"
    combined.save(os.path.join(output_folder, output_name))

    # === Track info for output file ===
    records.append({
        'Design': row['Design'],
        'Numer/Position_0': row['Numer/Position_0'],
        'Type_0': row['Type_0'],
        'Size_0': row['Size_0'],
        'Color_0': row['Color_0'],
        'Number/Position_1': row['Number/Position_1'],
        'Type_1': row['Type_1'],
        'Size_1': row['Size_1'],
        'Color_1': row['Color_1'],
        'File_Name': output_name,
        'Manual_Correct_Choice': row['New Correct Choice'],
        'NPZ_Correct_Choice': target_plus_one
    })

In [8]:
# === Save to Excel ===
if records:
    final_df = pd.DataFrame(records)
    output_excel_path = os.path.join(output_folder, 'result_df_selected.xlsx')
    final_df.to_excel(output_excel_path, index=False)
    print(f"✅ Cross-check file saved to: {output_excel_path}")
else:
    print("⚠️ No records processed. Check if New_Item values exist and paths are correct.")

✅ Cross-check file saved to: C:\Users\dinos\PycharmProjects\refactored_SRAN\image\result_df_selected.xlsx
